In [1]:
# Data Downloaded From: https://www.kaggle.com/datasets/mathurinache/1000000-bandcamp-sales

# Importing Numerical Packages
import pandas as pd
import numpy as np

# Pandas Settings
pd.set_option('display.max_row', None)
pd.set_option('display.max_column', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

# Importing Visualization Packages
import matplotlib.pyplot as plt 
import seaborn as sns 

# For Handling Times
import pytz

# Importing CSV
df = pd.read_csv(r'C:\Users\nickb\Documents\SeattleU\Fall 2022\DataVisualization_BUAN5210\Final Project\1000000-bandcamp-sales.csv')
df.head()

,_id,art_url,item_type,utc_date,country_code,track_album_slug_text,country,slug_type,amount_paid_fmt,item_price,item_description,art_id,url,amount_paid,releases,artist_name,currency,album_title,amount_paid_usd,package_image_id,amount_over_fmt,item_slug,addl_count
0,1599688803.5175&//girlbanddublin.bandcamp.com/...,https://f4.bcbits.com/img/a0206405257_7.jpg,a,1599688803.517,gb,NaN,United Kingdom,a,$9.99,9.990,Live at Vicar Street,206405257.000,//girlbanddublin.bandcamp.com/album/live-at-vi...,9.990,NaN,Girl Band,USD,NaN,9.990,NaN,NaN,NaN,NaN
1,1599688805.27838&//maharettarecords.bandcamp.c...,https://f4.bcbits.com/img/a2984241552_7.jpg,a,1599688805.278,fi,NaN,Finland,a,£1,1.000,Neurogen,2984241552.000,//maharettarecords.bandcamp.com/album/neurogen,1.000,NaN,Jirah,GBP,NaN,1.300,NaN,NaN,NaN,NaN
2,1599688805.90646&//maharettarecords.bandcamp.c...,https://f4.bcbits.com/img/a3320494770_7.jpg,a,1599688805.906,fi,NaN,Finland,a,£3,3.000,The Last Snare Bender,3320494770.000,//maharettarecords.bandcamp.com/album/the-last...,3.000,NaN,D-Ther,GBP,NaN,3.900,NaN,NaN,NaN,NaN
3,1599688806.94234&//alicesitski.bandcamp.com/al...,https://f4.bcbits.com/img/0020476345_37.jpg,p,1599688806.942,gb,NaN,United Kingdom,a,€10.50,10.500,Limited Edition Compact Disc,NaN,//alicesitski.bandcamp.com/album/white-noise-tv,10.500,NaN,WHITE NOISE TV,EUR,WHITE NOISE TV,12.390,20476345.000,NaN,NaN,NaN
4,1599688809.07942&//linguaignota.bandcamp.com/t...,https://f4.bcbits.com/img/a3428873396_7.jpg,t,1599688809.079,us,NaN,United States,t,$1,1.000,O Ruthless Great Divine Director,3428873396.000,//linguaignota.bandcamp.com/track/o-ruthless-g...,1.000,NaN,LINGUA IGNOTA,USD,NaN,1.000,NaN,NaN,NaN,NaN


In [2]:
# Replacing Item Names
replace_item_type = {'a': 'Digital Album',
'p': 'Physical Item',
't': 'Digital Track'}
df = df.replace({"item_type": replace_item_type})

# Renaming Columns for Clarity
df.rename(columns = {'country':'Buyer Country',
                     'releases':'Total Artist Releases'}, inplace = True)

# Dropping Duplicate or Irrelevant Columns
df.drop(['track_album_slug_text', 'country_code', 'art_id', '_id', 'art_url', 'url',
         'package_image_id', 'Total Artist Releases', 'item_slug'], axis=1, inplace=True)

# Renaming Columns for Clarity
replace_dict = {'t': 'Digital Track', 'a': 'Digital & Physical Albums', 'p': 'Merchandise'}
df['slug_type'].replace(replace_dict, inplace=True)


# This is how I found out that b is equal to 'Full Digital Discography'. Same Descriptions more or less. Uncomment to run if you're curious.
# full_dig = df.loc[(df['item_type'] == 'b')]
# full_dig['item_description'].value_counts()

# Imputting Value for 'Full Digital Discography'
df.loc[(df['item_type'] == 'b'), 'item_type'] = 'Full Digital Discography'

# Imputting Value for Physical Album
df.loc[(df['item_type'] != 'Digital Album') & (df['slug_type'] == 'Digital & Physical Albums'), 'item_type'] = 'Physical Album'

# Creating (Hopefully) Clearer Feature Names
df.rename(columns={'amount_paid_fmt': 'Currency Symbol + Paid (In Seller Currency)',
                   'item_price': 'Price (Seller Currency)',
                   'amount_paid': 'Paid (Seller Currency)',
                   'amount_paid_usd': 'Paid (US Dollars)',
                   'slug_type': 'Item Category (Main)',
                   'item_type': 'Item Category (Subcategory)',
                   'amount_over_fmt': 'Paid OVER List Price (Seller Currency)',
                   'artist_name': 'Artist/Label Name'
                   }, inplace=True)

# Creating a New Column that Returns 'Yes' if the input value is less than 0.000001, indicating that the price is effectively zero, otherwise returns 'No'.
def no_price (value):
   if value < 0.000001: 
      return 'Yes' # Done as Yes/No, so it doesn't auto turn this into a numerical measure
   return 'No'

df['Item Listed As Free'] = df['Price (Seller Currency)'].map(no_price)
df['Item Listed As Free'].value_counts()


No     879253
Yes    120747
Name: Item Listed As Free, dtype: int64

In [3]:
# Remove Rows Without Artist/Label Name
df = df[df['Artist/Label Name'].notna()]

In [4]:
# Converting Epoch Time to Time
df['utc_date'] = pd.to_datetime(df['utc_date'],unit='s')

# Converting Time to Pacific (Timezone Bandcamp Uses For Bandcamp Fridays)
df['utc_date'] = df['utc_date'].dt.tz_localize('US/Pacific').dt.tz_convert('UTC')

# Renaming Date as it's now Pacific Timezone
df = df.rename({'utc_date': 'Date_Time_PT',}, axis=1) 

# Creating New Column for the Percentage a Buyer Paid Over the Seller's List Price on an Item
df['Percent Paid Over List Price'] = ((df['Paid (Seller Currency)'] - df['Price (Seller Currency)']) / df['Price (Seller Currency)'])

In [5]:
# Replacing -0.000 and 0.000 with 0
df['Percent Paid Over List Price'] = df['Percent Paid Over List Price'].replace([-0.000, 0.000], 0)

In [6]:
# Create a New Column Called Discount Code Applied if the Percent over List Price is less than 0, which assigns a value of Yes in that case
def discount_code (value):
    if value < -0.00001: 
        return 'Yes'
    return 'No'

df['Discount Code Applied'] = df['Percent Paid Over List Price'].map(discount_code)

In [7]:
# Rename Currency to Seller Currency
df.rename(columns={'currency': 'Seller Currency'}, inplace=True)

# Reordering Dataframe
df = df[['Date_Time_PT', 'Item Category (Main)', 'Item Category (Subcategory)', 'item_description','Artist/Label Name', 'Buyer Country', 'Seller Currency','Currency Symbol + Paid (In Seller Currency)', 'Price (Seller Currency)', 'Paid (Seller Currency)', 'Paid (US Dollars)', 'Paid OVER List Price (Seller Currency)', 'Item Listed As Free', 'Percent Paid Over List Price']]


In [8]:
# Over 100k of items had isna for paid over list price item, so recalculating colum. Uncomment next line to see.
# df['Paid OVER List Price (Seller Currency)'].isna().value_counts()
df['Paid OVER List Price (Seller Currency)'] = (df['Paid (Seller Currency)'] - df['Price (Seller Currency)'])

# Rounding to Two Decimal Places, it seemed like some columns were not binning values correctly
df['Paid OVER List Price (Seller Currency)'] = df['Paid OVER List Price (Seller Currency)'].round(decimals = 2)

# Replace all the NaN Values which based on other columns can tell are 'Full Digital Discography" with that
df['Item Category (Main)'] = df['Item Category (Main)'].replace(np.NaN,'Full Digital Discography')

# Gave More Accurate Name. There are EP's & Singles Shown Too.
df.rename(columns={'album_title': 'Release Title'
                   }, inplace=True)

df['Percent Paid Over List Price'] = df['Percent Paid Over List Price'].round(decimals = 2)

# Replacing Inifinite Values with NaN. This Way in Tableau, It Shows Datatype as Numeric
df['Percent Paid Over List Price'] = df['Percent Paid Over List Price'].replace(np.inf, np.NaN)

df['Bandcamp_Friday?'] = df['Date_Time_PT']
# If on this date, then assign as Bandcamp Friday
df.loc[(df['Date_Time_PT'] > '2020-10-02', "Bandcamp_Friday?")] = "Yes"

# If on this date, then assign as NOT Bandcamp Friday
df.loc[(df['Date_Time_PT'] < '2020-10-02', "Bandcamp_Friday?")] = "No"

# Renaming to a more straightforward title
df['Item Category (Main)'] = df['Item Category (Main)'].replace('Digital & Physical Albums', 'Albums & EP\'s')

# Replaces the value 'Digital & Physical Albums' with 'Albums & EP's' in the 'Item Category (Main)' column of the DataFrame 'df'
df['Item Category (Main)'] = df['Item Category (Main)'].replace('Digital & Physical Albums', 'Albums & EP\'s')

Inside of item description there is a lot of information, that could be changed into it's own column.

Also during this phase I found that a lot of albums that labels and artists had categorized as merchandise was actually physical releases. I was able to identify all of these different categories by slowly iterating through Merchandise that was classified as 'Other Merch' and seeing if I could identify larger categories. For example, 'Face Masks' were found as a result of doing this.

In [9]:
# This Code Could Definitely be Cleaner, but it works. It's a bit of a mess, but it works.

conditions = [
    (df['Item Category (Subcategory)'] == 'Digital Album') | (df['Item Category (Subcategory)'] == 'Digital Track') | (df['Item Category (Subcategory)'] == 'Full Digital Discography'),
    (df['Item Category (Main)'] == 'Merchandise') & (df['item_description'].str.contains('shirt', case=False, regex=True))
                                                  | (df['item_description'].str.contains('Tee', case=False, regex=True))
                                                  | (df['item_description'].str.contains('Baseball T', case=False, regex=True)), # Shirt
    (df['Item Category (Main)'] == 'Merchandise') & (df['item_description'].str.contains('hoodie', case=False, regex=True))
                                                  | (df['item_description'].str.contains('sweater', case=False, regex=True)), # Hoodie
    (df['Item Category (Main)'] == 'Merchandise') & (df['item_description'].str.contains('Tote', case=False, regex=True)), # Tote Bag
    (df['Item Category (Main)'] == 'Merchandise') & (df['item_description'].str.contains('Patch', case=False, regex=True)), # Patches
    (df['Item Category (Main)'] == 'Merchandise') & (df['item_description'].str.contains('Mask', case=False, regex=True)), # Mask
    (df['Item Category (Main)'] == 'Merchandise') & (df['item_description'].str.contains('Bundle', case=False, regex=True))
                                                  | (df['item_description'].str.contains('Pack', case=False, regex=True)) # Bundle... might contain vinyl too
                                                  | (df['item_description'].str.contains(' Box', case=False, regex=True)) # Bundle... might contain vinyl too
                                                  | (df['item_description'].str.contains('Swag', case=False, regex=True)), # Bundle... might contain vinyl too
    (df['Item Category (Main)'] == 'Merchandise') & (df['item_description'].str.contains('Cassette', case=False, regex=True))
                                                  | (df['item_description'].str.contains('Tape', case=False, regex=True)), # Cassette
    (df['Item Category (Main)'] == 'Merchandise') & (df['item_description'].str.contains('10"', case=False, regex=True))
                                                  | (df['item_description'].str.contains('7"', case=False, regex=True))
                                                  | (df['item_description'].str.contains(' LP', case=False, regex=True))
                                                  | (df['item_description'].str.contains('12"', case=False, regex=True))
                                                  | (df['item_description'].str.contains('Repress', case=False, regex=True))
                                                  | (df['item_description'].str.contains('LP', case=False, regex=True))
                                                  | (df['item_description'].str.contains('Vinyl', case=False, regex=True)), # Vinyl
    (df['Item Category (Main)'] == 'Merchandise') & (df['item_description'].str.contains('Sticker', case=False, regex=True)), # Sticker
    (df['Item Category (Main)'] == 'Merchandise') & (df['item_description'].str.contains('Longsleeve', case=False, regex=True))
                                                  | (df['item_description'].str.contains('Long-Sleeve', case=False, regex=True))
                                                  | (df['item_description'].str.contains('Long Sleeve', case=False, regex=True)), # Long-Sleeves
    (df['Item Category (Main)'] == 'Merchandise') & (df['item_description'].str.contains('Poster', case=False, regex=True))
                                                  | (df['item_description'].str.contains('Print', case=False, regex=True)), # Poster
    (df['Item Category (Main)'] == 'Merchandise') & (df['item_description'].str.contains('Ticket', case=False, regex=True)), # Ticket
    (df['Item Category (Main)'] == 'Merchandise') & (df['item_description'].str.contains('Book', case=False, regex=True))
                                                  | (df['item_description'].str.contains('Zine', case=False, regex=True))
                                                  | (df['item_description'].str.contains('Paperback', case=False, regex=True)), # Books
    (df['Item Category (Main)'] == 'Merchandise') & (df['item_description'].str.contains('CD', case=False, regex=True)) # CD
                                                  | (df['item_description'].str.contains('Compact Disc', case=False, regex=True)), 
    (df['Item Category (Main)'] == 'Merchandise') & (df['item_description'].str.contains('Trucker Hat', case=False, regex=True)) |
                                                    (df['item_description'].str.contains('Baseball Hat', case=False, regex=True)) |
                                                    (df['item_description'].str.contains('hat', case=False, regex=True)) |
                                                    (df['item_description'].str.contains('Beanie', case=False, regex=True)) |
                                                    (df['item_description'].str.contains(' cap', case=False, regex=True)), # Hat
    (df['Item Category (Main)'] == 'Merchandise') & (df['item_description'].str.contains('Button', case=False, regex=True)) |
                                                    (df['item_description'].str.contains('Pin', case=False, regex=True)),
    (df['Item Category (Main)'] == 'Merchandise') & (df['item_description'].str.contains('Pants', case=False, regex=True)),
    (df['Item Category (Main)'] == 'Merchandise') & (df['item_description'].str.contains('Towel', case=False, regex=True)),
    (df['Item Category (Main)'] == 'Merchandise') & (df['item_description'].str.contains('Scarf', case=False, regex=True)),
    (df['Item Category (Main)'] == 'Merchandise') & (df['item_description'].str.contains('Robe', case=False, regex=True)),
    (df['Item Category (Main)'] == 'Merchandise')] # Other Merch

choices = ['Digital Music','Shirt', 'Hoodie','Tote Bag','Patch','Face Mask','Bundle','Cassette','Vinyl','Sticker','Long-Sleeve Shirts','Poster/Print','Tickets','Book','CD','Hats','Buttons/Pins','Pants','Towel','Scarves','Robes','Other Items']
df['Merch_Category'] = np.select(conditions, choices, default='Not Merch')

# Check item_description column and if it contains 'Stream' then assign 'Live Stream' to Merch_Category
df['Merch_Category'] = np.where((df['item_description'].str.contains('live stream', case=False, regex=True)), 'Live Stream', df['Merch_Category'])


### Identifying Merchandise Items

In [10]:
# Create a new dataframe for Other Items
df_other_items = df[(df['Merch_Category'] == 'Other Items')]

While at first I was manually iterating through the dataframe and assessing new categories of items, this was taking longer than expected and there were still 10k items without clear categories. As such, I implemented Natural Language Processing to assist with further processing.
While some values were found that were inconclusive such as 'Limited', which could apply to a variety of items, others such as 'Pack' and 'Bottle' had clearer categories. 

In [11]:
import pandas as pd
import nltk
# nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist

stopwords = nltk.corpus.stopwords.words('english')
df_other_items['Merch_Category_Processed'] = df_other_items['item_description'].apply(lambda x: ' '.join([word.lower() for word in word_tokenize(x) if (word.isalpha()) and (word.lower() not in stopwords)]))

fdist = FreqDist()
for row in df_other_items['item_description']:
    for word in row.split():
        fdist[word] += 1

top_words = fdist.most_common(100)
print(top_words)

[('-', 1805), ('The', 502), ('Edition', 438), ('Limited', 418), ('Live', 406), ('of', 381), ('the', 355), ('/', 323), ('&', 288), ('A', 285), ('2020', 265), ('USB', 246), ('by', 239), ('THE', 231), ('BOTTLE', 222), ('OPENER', 222), ('Your', 218), ('Logo', 197), ('Matmos', 191), ('Low-Cost-co', 191), ('Stream', 191), ('Dream!', 191), ('from', 184), ('Camiseta', 183), ('+', 182), ('Signed', 182), ('From', 172), ('Mug', 171), ('for', 156), ('Issue', 152), ('Cheki', 152), ('T', 152), ('OF', 148), ('Batch', 141), ('2', 137), ('to', 129), ('Private', 126), ('OUT', 122), ('|', 121), ('with', 120), ('DARKNESS', 120), ('*Pre-Order*', 117), ('Document', 117), ('Culture', 117), ('Pamphlet', 117), ('CXT002', 117), ('on', 116), ('x', 115), ('VOTE!', 115), ('BIDEN', 115), ('HARRIS', 115), ('Black', 114), ('Joan', 114), ('Shelley', 114), ('Camp', 114), ('Ample', 114), ('Branch', 114), ('1', 112), ('PREORDER', 111), ('I', 108), ('New', 107), ('DVD', 105), ('Music', 104), ('Fall', 102), ('Flag', 100), 

C:\Users\nickb\AppData\Local\Temp\ipykernel_41840\820680613.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_other_items['Merch_Category_Processed'] = df_other_items['item_description'].apply(lambda x: ' '.join([word.lower() for word in word_tokenize(x) if (word.isalpha()) and (word.lower() not in stopwords)]))


In [12]:
# Check item_description for na values
df['item_description'].fillna('No Description', inplace=True)

In [13]:
# Show only  the rows where the item category is 'Merchandise' and the merch Category is Vinyl. Take these rows and change the item category main to Albums & EP's
df.loc[(df['Item Category (Main)'] == 'Merchandise') & (df['Merch_Category'] == 'Vinyl'), 'Item Category (Main)'] = 'Albums & EP\'s'

In [14]:
# Create a new column called New_Main_Category and set it to the value of Item Category (Main)
df['New_Main_Category'] = df['Item Category (Main)']

# Take all the Digital Tracks and Full Digital Disography and Album & EP's and change the New_Main_Category to Digital Music
df.loc[(df['Item Category (Main)'] == 'Digital Tracks') | (df['Item Category (Main)'] == 'Full Digital Discography') | (df['Item Category (Main)'] == 'Albums & EPs'), 'New_Main_Category'] = 'Digital Music'

# Select Only Rows that For Item Subcategory is Digital Album and rename the New_Main_Category for these rows to Digital Music
df.loc[(df['Item Category (Subcategory)'] == 'Digital Album'), 'New_Main_Category'] = 'Digital Music'

# Select Only Rows that For Item Subcategory is Digital Track and rename the New_Main_Category for these rows to Digital Music
df.loc[(df['Item Category (Subcategory)'] == 'Digital Track'), 'New_Main_Category'] = 'Digital Music'

# Select Only Rows that For Item Subcategory is Physical Album and rename the New_Main_Category for these rows to Physical Music
df.loc[(df['Item Category (Subcategory)'] == 'Physical Album'), 'New_Main_Category'] = 'Physical Music'

# Show Only Rows that for Merch_Category has Vinyl and rename the new_main_category to Physical Music
df.loc[(df['Merch_Category'] == 'Vinyl'), 'New_Main_Category'] = 'Physical Music'

# Show only rows that in item_desription "digipack" or "Digipak" is present and doesn't say "Bundle" and for Merch_Category make it say CD
df.loc[(df['item_description'].str.contains('digipack', case=False, regex=True)) | (df['item_description'].str.contains('Digipak', case=False, regex=True)) & (df['Merch_Category'] != 'Bundle'), 'Merch_Category'] = 'CD'

# Show only rows that for merch category is 'Tickets' or 'Live Stream' and rename the new main category to 'Tickets'
df.loc[(df['Merch_Category'] == 'Tickets') | (df['Merch_Category'] == 'Live Stream'), 'New_Main_Category'] = 'Tickets'

# Rename New_Main_Category to New_Category_Level_1
df.rename(columns={'New_Main_Category': 'New_Category_Level_1'}, inplace=True)

### Populating New_Category_Level_2

In [15]:
# Create a New Column Called New_Category_Level_2 and fill with null values
df['New_Category_Level_2'] = np.nan

# Create a new column called 'New_Category_Level_3' and set all the values to null
df['New_Category_Level_3'] = np.nan

In [16]:
# If in Item Category (Subcategory) is 'Digital Album', then set New_Category_Level_2 to 'Albums & EPs'
df.loc[(df['Item Category (Subcategory)'] == 'Digital Album'), 'New_Category_Level_2'] = 'Albums & EPs'

# If in Item Category (Subcategory) is 'Digital Track', then set New_Category_Level_2 to 'Individiual Track'
df.loc[(df['Item Category (Subcategory)'] == 'Digital Track'), 'New_Category_Level_2'] = 'Individiual Track'

# If in Item Category (Subcategory) is 'Digital Discography', then set New_Category_Level_2 to 'Full Digital Discography'
df.loc[(df['Item Category (Subcategory)'] == 'Digital Discography'), 'New_Category_Level_2'] = 'Full Digital Discography'

# If in Merch Category is 'CD', then set New_Category_Level_2 to 'CD'
df.loc[(df['Merch_Category'] == 'CD'), 'New_Category_Level_2'] = 'CD'

#If in Merch Category is 'Vinyl', then set New_Category_Level_2 to 'Vinyl'
df.loc[(df['Merch_Category'] == 'Vinyl'), 'New_Category_Level_2'] = 'Vinyl'

#If in Merch Category is 'Shirt' or 'Long-Sleeve Shirts' then set New_Category_Level_2 to 'Tops'
df.loc[(df['Merch_Category'] == 'Shirt') | (df['Merch_Category'] == 'Long-Sleeve Shirts'), 'New_Category_Level_2'] = 'Tops'

# If in Merch Category is 'Hoodie' or 'Sweatshirt' then set New_Category_Level_2 to 'Outerwear'
df.loc[(df['Merch_Category'] == 'Hoodie') | (df['Merch_Category'] == 'Sweatshirt'), 'New_Category_Level_2'] = 'Outerwear'

#If in Merch Category is 'Hat' or 'Beanie' or 'Patch' or 'Tote Bag' then set New_Category_Level_2 to 'Accessories'
df.loc[(df['Merch_Category'] == 'Hat') | (df['Merch_Category'] == 'Beanie') | (df['Merch_Category'] == 'Patch') | (df['Merch_Category'] == 'Tote Bag'), 'New_Category_Level_2'] = 'Accessories'

#If in Merch Category is 'Patch' or 'BUttons or Pins or Face Masks then set New_Category_Level_2 to 'Accessories'
df.loc[(df['Merch_Category'] == 'Patch') | (df['Merch_Category'] == 'Buttons/Pins') | (df['Merch_Category'] == 'Face Mask'), 'New_Category_Level_2'] = 'Accessories'

# If IN Merch Category its Poster/Print or Book then set New_Category_Level_2 to 'Prints & Books'
df.loc[(df['Merch_Category'] == 'Poster/Print') | (df['Merch_Category'] == 'Book'), 'New_Category_Level_2'] = 'Prints & Books'

# If IN Merch Category its Towel then set New_Category_Level_2 to Accessories
df.loc[(df['Merch_Category'] == 'Towel'), 'New_Category_Level_2'] = 'Accessories'

# If in Item Category (Subcategory) is 'Full Digital Discography' then set New_Category_Level_2 to 'Full Digital Discography'
df.loc[(df['Item Category (Subcategory)'] == 'Full Digital Discography'), 'New_Category_Level_2'] = 'Full Digital Discography'

#If in In item_description the word 'Cassette' is present then set New_Category_Level_2 to 'Cassette'
df.loc[(df['item_description'].str.contains('Cassette', case=False, regex=True)), 'New_Category_Level_2'] = 'Cassette'

#If in In item_description the word 'CD' is present then set New_Category_Level_2 to 'CD'
df.loc[(df['item_description'].str.contains('CD', case=False, regex=True)), 'New_Category_Level_2'] = 'CD\'s'

# If in Item_Descrption 7” or 10” or 12” is present then set New_Category_Level_2 to 'Vinyl'
df.loc[(df['item_description'].str.contains('7"', case=False, regex=True)) | (df['item_description'].str.contains('10"', case=False, regex=True)) | (df['item_description'].str.contains('12"', case=False, regex=True)), 'New_Category_Level_2'] = 'Vinyl'

#If in item_description the phrase "Picture Disc" is present then set New_Category_Level_2 to 'Vinyl'
df.loc[(df['item_description'].str.contains('Picture Disc', case=False, regex=True)), 'New_Category_Level_2'] = 'Vinyl'

# If in item_description "3xLP" or "2xLP" or "4xLP" or "5xLP" or "6xLP" or "7xLP" or "8xLP" or "9xLP" or "10xLP" is present then set New_Category_Level_2 to 'Vinyl'
df.loc[(df['item_description'].str.contains('3xLP', case=False, regex=True)) | (df['item_description'].str.contains('2xLP', case=False, regex=True)) | (df['item_description'].str.contains('4xLP', case=False, regex=True)) | (df['item_description'].str.contains('5xLP', case=False, regex=True)) | (df['item_description'].str.contains('6xLP', case=False, regex=True)) | (df['item_description'].str.contains('7xLP', case=False, regex=True)) | (df['item_description'].str.contains('8xLP', case=False, regex=True)) | (df['item_description'].str.contains('9xLP', case=False, regex=True)) | (df['item_description'].str.contains('10xLP', case=False, regex=True)), 'New_Category_Level_2'] = 'Vinyl'

# If "earrings" appears in the item_description then for New_Category_Level_2 set to 'Accessories'
df.loc[(df['item_description'].str.contains('earrings', case=False, regex=True)), 'New_Category_Level_2'] = 'Accessories'

# where item_description contains Ableton then set New_Category_Level_2 to 'Merchandise' and New_Category_level_3 to 'Music Software Plug-In\s'
df.loc[(df['item_description'].str.contains('Ableton', case=False, regex=True)), 'New_Category_Level_2'] = 'Merchandise'
df.loc[(df['item_description'].str.contains('Ableton', case=False, regex=True)), 'New_Category_Level_3'] = 'Music Software Plug-In\s'

In [17]:
# rows where item_description contains "10 inch" or "7 inch" or "12 inch" and classify New_Category_Level_2 as 'Vinyl'
df.loc[(df['item_description'].str.contains('10 inch', case=False, regex=True)) | (df['item_description'].str.contains('7 inch', case=False, regex=True)) | (df['item_description'].str.contains('12 inch', case=False, regex=True)), 'New_Category_Level_2'] = 'Vinyl'

# rows where item category (subcategory) is 'Physical Album' and 'New_categry_level_2' is null amd Merch Category is 'Cassette' and classify New_Category_Level_2 as 'Cassette'
df.loc[(df['Item Category (Subcategory)'] == 'Physical Album') & (df['New_Category_Level_2'].isna()) & (df['Merch_Category'] == 'Cassette'), 'New_Category_Level_2'] = 'Cassette'

# rows where item category (subcategory) is 'Physical Album' and 'New_categry_level_2' is null amd Merch Category is 'Hats' and classify New_Category_Level_2 as 'Accessories'
df.loc[(df['Item Category (Subcategory)'] == 'Physical Album') & (df['New_Category_Level_2'].isna()) & (df['Merch_Category'] == 'Hats'), 'New_Category_Level_2'] = 'Accessories'

# rows where item category (subcategory) is 'Physical Album' and 'New_categry_level_2' is null amd Merch Category is 'Bundle' and 'item_descrption' does not contain 'featuring' or 'CD' or 'with' or'VHS' or "+" or "inclujding" and classify New_Category_Level_2 as 'Vinyl'
df.loc[(df['Item Category (Subcategory)'] == 'Physical Album') & (df['New_Category_Level_2'].isna()) & (df['Merch_Category'] == 'Bundle') & (df['item_description'].str.contains('featuring', case=False, regex=True) == False) & (df['item_description'].str.contains('CD', case=False, regex=True) == False) & (df['item_description'].str.contains('with', case=False, regex=True) == False) & (df['item_description'].str.contains('VHS', case=False, regex=True) == False) & (df['item_description'].str.contains('\+', case=False, regex=True) == False) & (df['item_description'].str.contains('including', case=False, regex=True) == False), 'New_Category_Level_2'] = 'Vinyl'

# Get a count of the rows where in item_description the word "flannel" and either "large" or "medium" or "small" or "XL" or "XS" is present and for New_Category_Level_2 set to 'Tops'
df.loc[(df['item_description'].str.contains('flannel', case=False, regex=True)) & (df['item_description'].str.contains('large', case=False, regex=True)) | (df['item_description'].str.contains('medium', case=False, regex=True)) | (df['item_description'].str.contains('small', case=False, regex=True)) | (df['item_description'].str.contains('XL', case=False, regex=True)) | (df['item_description'].str.contains('XS', case=False, regex=True)), 'New_Category_Level_2'] = 'Tops'

# for Rows where in item_description the phrase "TEST PRESSING" is present and set New_Category_Level_2 to 'Vinyl'
df.loc[(df['item_description'].str.contains('TEST PRESSING', case=False, regex=True)), 'New_Category_Level_2'] = 'Vinyl'

# Get a count of rows where Double 45 is present in item_description and set New_Category_Level_2 to 'Vinyl'
df.loc[(df['item_description'].str.contains('Double 45', case=False, regex=True)), 'New_Category_Level_2'] = 'Vinyl'

# Get a Count of rows where in item_description the word " sticker" is found
df.loc[(df['item_description'].str.contains('sticker', case=False, regex=True)), 'New_Category_Level_2'] = 'Accessories'

# Get the count of the number of rows where in item_description the word " sticker" is found but the symbols + or & or the words "and" or "pack" or "bundle" are not found and set New_Category_Level_2 to 'Accessories'
df.loc[(df['item_description'].str.contains('sticker', case=False, regex=True)) & ~(df['item_description'].str.contains('\+', case=False, regex=True)) & ~(df['item_description'].str.contains('&', case=False, regex=True)) & ~(df['item_description'].str.contains('and', case=False, regex=True)) & ~(df['item_description'].str.contains('pack', case=False, regex=True)) & ~(df['item_description'].str.contains('bundle', case=False, regex=True)), 'New_Category_Level_2'] = 'Accessories'

# Show only the rows where New_Category_Level_2 is null and in item_descrption the words "pack or "bundle" are present and for New_Category_Level_2 set to 'Bundles'
df.loc[(df['New_Category_Level_2'].isna()) & (df['item_description'].str.contains('pack', case=False, regex=True)) | (df['item_description'].str.contains('bundle', case=False, regex=True)), 'New_Category_Level_2'] = 'Bundles'

# If " Tape" appears in the item_description then for New_Category_Level_2 set to 'Cassettes'
df.loc[(df['item_description'].str.contains(' Tape', case=False, regex=True)), 'New_Category_Level_2'] = 'Cassettes'

# Show Only Rows where  C31	is present in item_description and assign new_main_category 2 to 'Cassette'
df.loc[(df['item_description'].str.contains('C31', case=False, regex=True)), 'New_Category_Level_2'] = 'Cassette'

# Create an empty column called 'New Category Level 4'
df['New_Category_Level_4'] = ''


### Creating a Column for the Artist or Label's Currency
This will be useful later when merging on the Discogs database

In [18]:
currency_country_dict = {
    'USD': 'United States',
    'GBP': 'United Kingdom',
    'EUR': 'European Union',
    'AUD': 'Australia',
    'JPY': 'Japan',
    'SEK': 'Sweden',
    'CAD': 'Canada',
    'CZK': 'Czech Republic',
    'NZD': 'New Zealand',
    'DKK': 'Denmark',
    'CHF': 'Switzerland',
    'ILS': 'Israel',
    'HUF': 'Hungary',
    'PLN': 'Poland',
    'NOK': 'Norway',
    'MXN': 'Mexico',
    'HKD': 'Hong Kong',
    'SGD': 'Singapore'
}

# function to map currency to country
def map_currency_to_country(currency):
    if currency in currency_country_dict:
        return currency_country_dict[currency]
    else:
        return ''

# create a new column called "Artist/Label Country"
df['Artist/Label Country'] = ''

# iterate through each row and map currency to country
for index, row in df.iterrows():
    currency = row['Seller Currency']
    country = map_currency_to_country(currency)
    df.at[index, 'Artist/Label Country'] = country

# Create a New Column Called 'Buyer/Seller Country Match' and if for a given row the artist/label country is equal to the Buyer Country then set it to yes
df['Buyer/Seller Country Match'] = np.where(df['Artist/Label Country'] == df['Buyer Country'], 'Yes', 'No')

In [19]:
# Show Only the Count of Distinct Artists/Labels where New_Category_Level_1 is 'Merchandise'
distinct_artists_labels_selling_merch = df['Artist/Label Name'].loc[(df['New_Category_Level_1'] == 'Merchandise')].nunique()

# Count of the Distinct Artist/Label Names
distinct_artists_labels = df['Artist/Label Name'].nunique()
print("Number of distinct values in 'Artist/Label Name': ", distinct_artists_labels)

print("Only ",round((distinct_artists_labels_selling_merch / distinct_artists_labels) * 100,2),"% of the artists and labels on the site are selling merchandise.")

Number of distinct values in 'Artist/Label Name':  159747
Only  4.96 % of the artists and labels on the site are selling merchandise.


In [20]:
# # For rows where in merch_category it says 'Tote Bag', 'Face Mask','Hats', 'Sticker', 'Book', 'Buttons/Pins', 'Towel', 'Scarves', 'Pants', assign those column values to New_Category_Level_3 for that row
df.loc[(df['Merch_Category'] == 'Tote Bag'), 'New_Category_Level_3'] = 'Tote Bags'
df.loc[(df['Merch_Category'] == 'Face Mask'), 'New_Category_Level_3'] = 'Face Masks'
df.loc[(df['Merch_Category'] == 'Hats'), 'New_Category_Level_3'] = 'Hats'
df.loc[(df['Merch_Category'] == 'Sticker'), 'New_Category_Level_3'] = 'Stickers'
df.loc[(df['Merch_Category'] == 'Book'), 'New_Category_Level_3'] = 'Books'
df.loc[(df['Merch_Category'] == 'Buttons/Pins'), 'New_Category_Level_3'] = 'Buttons/Pins'
df.loc[(df['Merch_Category'] == 'Towel'), 'New_Category_Level_3'] = 'Towels'
df.loc[(df['Merch_Category'] == 'Scarves'), 'New_Category_Level_3'] = 'Scarves'
df.loc[(df['Merch_Category'] == 'Pants'), 'New_Category_Level_3'] = 'Pants'
df.loc[(df['Merch_Category'] == 'Patch'), 'New_Category_Level_3'] = 'Patches'
df.loc[(df['Merch_Category'] == 'Shirt'), 'New_Category_Level_3'] = 'Shirt'

#### Setting New_Category_Level 4

In [21]:
# Only Show Rows where New Category Level 3 is 'Shirt' and in item_description it says either 'T-Shirt' or 'Tee'
df.loc[(df['New_Category_Level_3'] == 'Shirt') & (df['item_description'].str.contains('T-Shirt', case=False, regex=True)) | (df['item_description'].str.contains('Tee', case=False, regex=True)) & ~(df['item_description'].str.contains('\+', case=False, regex=True)), 'New_Category_Level_4'] = 'T-Shirt'

# Only Show Rows where New Category Level 3 is 'Shirt' and New_Category_Level_4 is null
df.loc[(df['New_Category_Level_3'] == 'Shirt') & (df['New_Category_Level_4'].isna()), 'New_Category_Level_4'] = 'Other Shirts'

# If Merch_Catgory_Level_3 is tote bag then set New Category Level 4 to 'Tote Bag'
df.loc[(df['New_Category_Level_3'] == 'Tote Bags'), 'New_Category_Level_4'] = 'Tote Bags'

# If Merch_Catgory_Level_3 is Face Mask then set New Category Level 4 to 'Face Mask'
df.loc[(df['New_Category_Level_3'] == 'Face Masks'), 'New_Category_Level_4'] = 'Face Masks'

# If Merch_Catgory_Level_3 is Sticker then set New Category Level 4 to 'Sticker'
df.loc[(df['New_Category_Level_3'] == 'Stickers'), 'New_Category_Level_4'] = 'Stickers'

# If Merch_Catgory_Level_3 is Button then set New Category Level 4 to 'Button'
df.loc[(df['New_Category_Level_3'] == 'Buttons/Pins'), 'New_Category_Level_4'] = 'Buttons/Pins'

# If Merch_Catgory_Level_3 is Towel then set New Category Level 4 to 'Towel'
df.loc[(df['New_Category_Level_3'] == 'Towels'), 'New_Category_Level_4'] = 'Towels'

# If Merch_Catgory_Level_3 is Scarf then set New Category Level 4 to 'Scarf'    
df.loc[(df['New_Category_Level_3'] == 'Scarves'), 'New_Category_Level_4'] = 'Scarves'

# If Merch_Catgory_Level_3 is Pants then set New Category Level 4 to 'Pants'
df.loc[(df['New_Category_Level_3'] == 'Pants'), 'New_Category_Level_4'] = 'Pants'

# If Merch_Catgory_Level_3 is Patch then set New Category Level 4 to 'Patch'
df.loc[(df['New_Category_Level_3'] == 'Patches'), 'New_Category_Level_4'] = 'Patches'

### Creating Better Time Column for Tableau + Exporting To Tableau

In [22]:
# define a function to convert a single date to ISO 8601 format
def convert_to_iso(date):
    return date.isoformat()

# apply the function to each value in the 'date' column using apply()
df['iso_date'] = df['Date_Time_PT'].apply(convert_to_iso)

In [23]:
df['Merch_Category'].value_counts()

Digital Music         764110
Vinyl                 101878
Not Merch              63187
Shirt                  21955
CD                     17796
Other Items             9536
Bundle                  4797
Cassette                4597
Poster/Print            2048
Hoodie                  1673
Patch                   1412
Long-Sleeve Shirts      1373
Buttons/Pins            1370
Book                    1134
Hats                     906
Face Mask                732
Sticker                  655
Tote Bag                 433
Tickets                  242
Live Stream               71
Towel                     43
Scarves                   41
Robes                      6
Pants                      3
Name: Merch_Category, dtype: int64

In [24]:
# Show only rows where Merch_Category is Robes...
df.loc[(df['Merch_Category'] == 'Robes')]

,Date_Time_PT,Item Category (Main),Item Category (Subcategory),item_description,Artist/Label Name,Buyer Country,Seller Currency,Currency Symbol + Paid (In Seller Currency),Price (Seller Currency),Paid (Seller Currency),Paid (US Dollars),Paid OVER List Price (Seller Currency),Item Listed As Free,Percent Paid Over List Price,Bandcamp_Friday?,Merch_Category,New_Category_Level_1,New_Category_Level_2,New_Category_Level_3,New_Category_Level_4,Artist/Label Country,Buyer/Seller Country Match,iso_date
544,2020-09-10 05:16:36.737009920+00:00,Merchandise,Physical Item,Baba Naga Robe,Baba Naga,France,GBP,£10,10.000,10.000,12.990,0.000,No,0.000,No,Robes,Merchandise,NaN,NaN,,United Kingdom,No,2020-09-10T05:16:36.737009920+00:00
94398,2020-09-12 07:13:13.224069888+00:00,Merchandise,Physical Item,Spring - a poetry pamphlet by Fay Roberts,Allographic Press,United Kingdom,GBP,£5,3.500,5.000,6.400,1.500,No,0.430,No,Robes,Merchandise,NaN,NaN,,United Kingdom,Yes,2020-09-12T07:13:13.224069888+00:00
127585,2020-09-13 01:41:35.397959936+00:00,Merchandise,Physical Item,Strobe - deadmau5 (Guitar tab en pdf / .gp5),El Profe Mauro,Germany,USD,$5.99,5.990,5.990,5.990,0.000,No,0.000,No,Robes,Merchandise,NaN,NaN,,United States,No,2020-09-13T01:41:35.397959936+00:00
202929,2020-09-15 05:11:35.803020032+00:00,Merchandise,Physical Item,Strobe - deadmau5 (Guitar tab en pdf / .gp5),El Profe Mauro,United Kingdom,USD,$5.99,5.990,5.990,5.990,0.000,No,0.000,No,Robes,Merchandise,NaN,NaN,,United States,No,2020-09-15T05:11:35.803020032+00:00
702234,2020-09-27 04:58:28.875379968+00:00,Merchandise,Physical Item,Corridor by Jonathan Robert,Corridor,Canada,USD,$18,18.000,18.000,18.000,0.000,No,0.000,No,Robes,Merchandise,NaN,NaN,,United States,No,2020-09-27T04:58:28.875379968+00:00
930456,2020-10-02 15:53:07.162450176+00:00,Merchandise,Physical Item,Genuflection Robe Black/Gold,Baba Naga,United Kingdom,GBP,£15,15.000,15.000,19.290,0.000,No,0.000,Yes,Robes,Merchandise,NaN,NaN,,United Kingdom,Yes,2020-10-02T15:53:07.162450176+00:00


In [25]:
# If Merch_Category is Hats then set New Category Level 4 to 'Hats'
df.loc[(df['New_Category_Level_3'] == 'Hats'), 'New_Category_Level_4'] = 'Hats'

In [26]:
df['New_Category_Level_4'].value_counts()

                976460
T-Shirt          17943
Patches           1412
Buttons/Pins      1370
Hats               906
Face Masks         732
Stickers           655
Tote Bags          433
Towels              43
Scarves             41
Pants                3
Name: New_Category_Level_4, dtype: int64

In [27]:
df['Merch_Category'].value_counts()

Digital Music         764110
Vinyl                 101878
Not Merch              63187
Shirt                  21955
CD                     17796
Other Items             9536
Bundle                  4797
Cassette                4597
Poster/Print            2048
Hoodie                  1673
Patch                   1412
Long-Sleeve Shirts      1373
Buttons/Pins            1370
Book                    1134
Hats                     906
Face Mask                732
Sticker                  655
Tote Bag                 433
Tickets                  242
Live Stream               71
Towel                     43
Scarves                   41
Robes                      6
Pants                      3
Name: Merch_Category, dtype: int64

In [31]:
# Group by Artist Name and only show New_Category_Level 4 is 'T-Shirt' and do a describe on the Price Column
df.groupby('Artist/Label Name')['Paid (US Dollars)'].describe().loc[(df['New_Category_Level_4'] == 'T-Shirt')]

KeyboardInterrupt: 

In [28]:
# # Only Run When Ready to Export for Tableau
# df.to_csv('After_Python_Processing_bandcamp_million_sales.csv', header=True)

### Testing Exchange Rate Conversion

This can be tabled for now as we can just focus on US Dollar Transactions.

In [29]:
# # import pandas as pd

# # # create a sample table with some transactions
# # data = {
# #     "Seller Currency": ["EUR", "GBP", "CAD", "JPY"],
# #     "Price (Seller Currency)": [100, 50, 75, 10000],
# #     "Date_Time_PT": ["2022-03-30 10:00:00", "2022-03-30 11:00:00", "2022-03-29 12:00:00", "2022-03-29 13:00:00"],
# # }
# # test_df = pd.DataFrame(data)

# # Convert Date_Time_Pt to datetime
# test_df['Date_Time_PT'] = pd.to_datetime(test_df['Date_Time_PT'])

# import requests

# def get_exchange_rate(base_currency, target_currency, date):
#     api_url = f"https://api.exchangeratesapi.io/{date}"
#     response = requests.get(api_url, params={"base": base_currency, "symbols": target_currency})
#     if response.status_code == 200:
#         data = response.json()
#         if target_currency in data["rates"]:
#             return data["rates"][target_currency]
#     return None

# def convert_to_usd(row):
#     base_currency = row["Seller Currency"]
#     seller_price = row["Price (Seller Currency)"]
#     exchange_date = row["Date_Time_PT"].strftime("%Y-%m-%d")
#     exchange_rate = get_exchange_rate(base_currency, "USD", exchange_date)
#     if exchange_rate is not None:
#         return seller_price / exchange_rate
#     else:
#         return None

# # apply the function to create a new column with USD prices
# test_df["Price_USD"] = test_df.apply(convert_to_usd, axis=1)
